# Best Practices

We'll put into practice what we've learned so far by working with some real-world data. Let's begin with loading a log of sports activities of a single person:

In [ ]:
import pandas as pd

In [ ]:
activity_df = pd.read_json('data/activities.json', convert_dates=['start_date_local'])

## Some Simple and Useful DataFrame Methods

In [ ]:
activity_df.describe()

__Question__: What does this overview tell us at a first glance?

In [ ]:
activity_df.head()

In [ ]:
activity_df.tail(3)

In [ ]:
activity_df.sample(10)

In [ ]:
(
    activity_df
    .sort_values('start_date_local', ascending=False)
    .head(10)
)

In [ ]:
activity_df['type']

In [ ]:
activity_df['type'].unique()

In [ ]:
activity_df['type'].value_counts()

In [ ]:
len(activity_df)

__Question__: in evaluation of the above expression, is any `DataFrame` method called?

## Separation of Concerns

Assume we want to create a process that reads such activities from file, selects all the running activities, sorts these by date, and saves the result in another file. A possible implementation may look like this:

In [ ]:
def process_activities():
    activity_df = pd.read_json('data/activities.json', convert_dates=['start_date_local'])
    runs_sorted_by_date_df = (
        activity_df
        .loc[lambda df: df['type'] == 'Run']
        .sort_values('start_date_local')
    )
    runs_sorted_by_date_df.to_csv('data/processed_activities.csv')

In [ ]:
process_activities()

**Discussion**:

- What if there are many of these processes, and the data source changes from .json file to database?
- How can we test the business logic without access to a file system or database?
- How can this be improved?

In [ ]:
# !rm data/processed_activities.csv

As soon as our work shifts from pure exploratory analysis in a notebook to repeatable and robust production systems, separation of concerns becomes important. As discussed above, any code that combines business logic with side effects such as reading/writing data, handling user input from a GUI, etc, can easily break when there are any changes in infrastructure or user interaction.

The example in the cells below shows how our naive process implementation is improved by abstracting the infrastructure and separating the business logic from the process.

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Any, Callable

In [ ]:
# Infrastructure Layer
class ActivitySource(ABC):
    @abstractmethod
    def load(self):
        pass

@dataclass
class JsonLocalActivitySource(ActivitySource):
    filename: str = 'data/activities.json'  # will usually come from a config file
    
    def load(self):
        return pd.read_json(self.filename, convert_dates=['start_date_local'])

@dataclass
class PostgressActivitySource(ActivitySource):
    db_connection: Any  # will usually come from a config file
    
    def load(self):
        # pseudo-code
        return db_connection.query('SELECT * FROM activities')

class ActivitySink(ABC):
    @abstractmethod
    def save(self, activity_df: pd.DataFrame):
        pass

@dataclass
class CsvLocalActivitySink(ActivitySink):
    filename: str = 'data/processed_activities.csv'  # will usually come from a config file
    
    def save(self, activity_df: pd.DataFrame):
        activity_df.to_csv(self.filename)

In [ ]:
# Business Layer
def select_runs_and_sort_by_date(activity_df: pd.DataFrame) -> pd.DataFrame:
    return (
        activity_df
        .loc[lambda df: df['type'] == 'Run']
        .sort_values('start_date_local')
    )

In [ ]:
# Process Layer
def process_activities(
    source: ActivitySource,
    processing_fn: Callable[[pd.DataFrame], pd.DataFrame],
    sink: ActivitySink
) -> None:
    sink.save(processing_fn(source.load()))

In [ ]:
process_activities(JsonLocalActivitySource(), select_runs_and_sort_by_date, CsvLocalActivitySink())

Hint: for large/complex workflows, consider tools such as [Apache Airflow](https://airflow.apache.org/)

## Decluttering

In [ ]:
activity_df['year'] = activity_df['start_date_local'].dt.year
activity_df.head()

https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#dt-accessor
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [ ]:
activities_2018_df = activity_df.loc[activity_df['year'] == 2018]
activities_2018_by_type = activities_2018_df.groupby('type')

In [ ]:
mean_velocity_by_type = activities_2018_by_type['velocity_mean'].mean()
mean_velocity_by_type

In [ ]:
activity_df = pd.read_json('data/activities.json', convert_dates=['start_date_local'])
(
    activity_df
    .loc[lambda df: df['start_date_local'].dt.year == 2018]
    .groupby('type')['velocity_mean'].mean()
)

Introducing ... `Groupby.transform()`, TODO link to docs of agg/transform

In [ ]:
(
    activity_df
    .loc[lambda df: df['start_date_local'].dt.year == 2018]
    .groupby('type')['velocity_mean'].transform('mean')
)

In [ ]:
(
    activity_df
    .loc[lambda df: df['start_date_local'].dt.year == 2018]
    .assign(v_mean_diff=lambda df: (
        df.groupby('type')['velocity_mean']
        .transform(lambda type_v: type_v - type_v.mean())))
    .sample(10)
)

## Readable, Testable Pipelines

In [ ]:
def is_non_warmup(activity_df):
    return (
        (activity_df['elapsed_time'] > 600) |
        (activity_df['heartrate_mean'] > 150)
    )

def select_non_warmup_runs(activity_df):
    return (
        activity_df
        .loc[is_non_warmup]
        .loc[lambda df: df['type'] == 'Run']
    )

def to_z_score(series):
    # What if series has a lenght of 1?
    return (series - series.mean()) / series.std()

def add_z_score(activity_df, column):
    return activity_df.assign(**{f'{column}_z': lambda df: to_z_score(df[column])})

def best_n_years(activity_df, metrics=['velocity_mean'], n_years=1):
    return (
        activity_df
        .assign(year=lambda df: df['start_date_local'].dt.year)
        .groupby('year')[metrics].mean()
        .sort_values(metrics, ascending=False)
        .head(n_years)
    )

In [ ]:
(
    activity_df
    .pipe(select_non_warmup_runs)
    .pipe(add_z_score, column='velocity_mean')
    .pipe(best_n_years, metrics=['velocity_mean_z', 'velocity_mean'], n_years=5)
)

__Discussion__: Is this a reliable analysis? Which question does it answer? What can be improved?

## [Don't Repeat Yourself](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)

In [ ]:
import logging
import time
from functools import wraps

In [ ]:
logging.basicConfig(level=logging.DEBUG)  # You may need to restart the notebook to make this work
logger = logging.getLogger(__name__)

In [ ]:
logger.warning(f'There are {len(activity_df)} activities in the DataFrame')
logger.debug(f'If this line is not visible, restart the notebook kernel')

__Exercise__: Create a decorator that logs the number of rows and columns of the DataFrame that is passed to the pipeline functions above. You can use the `@skip_None` and `@check_value` decorators of the previous module as examples.

__Bonus Exercise__: Extend this decorator such that it also logs the same information about the DataFrame that is returned by the pipeline function.

__Bonus Exercise__: Extend this decorator such that it also logs how many rows are missing for each column. The [`isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html) method and `sum()` built-in function may be helpful for this purpose.

__Bonus Exercise__: Extend this decorator such that it also logs how much time it took to call the decorated pipeline function.

In [ ]:
# Your solution:

In [ ]:
# %load solutions/pipe_logging.py

## Bonus Exercises

What is the longest run per year?

In [ ]:
# Your solution:

In [ ]:
# %load solutions/longest_run_per_year.py

What is the average (mean) number of hours spent cycling by weekday? It is ok to ignore the days without activity (i.e. these don't need to be counted as 0 for the mean).

In [ ]:
# Your solution:

In [ ]:
# %load solutions/hours_cycling_by_weekday.py

What are the 10 longest time gaps between activities? The method [`diff()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html) may be useful for this purpose.

In [ ]:
# Your solution:

In [ ]:
# %load solutions/longest_time_gaps.py

What are the 7-day periods with the highest total amount of activity (in hours)? Similar to `groupby()`, Pandas has a method [`rolling()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) for using an (aggregating) sliding window. It can even use a flexible time-based window size, given that the DataFrame index is a datetime-like (and sorted).

In [ ]:
(
    pd.DataFrame({'a': range(5), 'b': range(5, 10)})
    .rolling(3).mean()
)

In [ ]:
(
    pd.DataFrame({'a': range(5), 'b': range(5, 10)})
    .rolling(3, min_periods=1).mean()
)

In [ ]:
# Your solution:

In [ ]:
# %load solutions/total_time_7days.py

Observe the 20 or 30 runs with highest mean heartrate. Looking at the name and distance, do you see some common pattern? Looking at other columns such as velocity_mean, do you see any activities that are out of the ordinary? Can it be that there are outliers based on measurement errors?

Try to create an analysis that detects runs with an uncommon combination of mean heartrate and mean velocity, for example, by comparing z values. If you're familiar with Scikit-Learn, consider using one of its outlier detection methods or fitting a (linear) regression and looking for large residuals.

In [ ]:
activity_df.sort_values('heartrate_mean', ascending=False).head(20)

In [ ]:
# Your solution: